#### Com base no exemplo do titanic iremos realizar alguns experimentos
   - Importe as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### Considere os seguintes algoritmos
   - Importe a classe [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) e [KNN Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) do Sklearn

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#### Realize o processo de treinamento e predição para o problema considerando um dos modelos sugeridos acima

In [12]:
df = pd.read_csv('titanic_ready_to_ml.csv')
X = df.iloc[:,1:]
y = df[['Survived']]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                    random_state=101)

In [18]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

#### Verifique a performance do algoritmo

### Grid Search e Cross Validation

#### Aplique apenas [CV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) e verifique se a métrica de avaliação considerada ainda permanece a mesma.

#### Selecione dois hiperparâmetros, discuta a importância destes com seu colega

#### Defina variações destes hiperparâmetros e aplique o processo de [Grid Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

#### Aplique Grid Search e cross validation. Por padrão o Grid Search já aplica CV. O objetivo deste exercício é você se familiarizar com as possibilidades existentes de experimentação. Leia o [ShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit) e [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html).
* *Desafio:* Aplique o shuffle considerando o balanceamento das classes

#### Quais os benefícios de utilizar as técnicas de CV e Grid Search na construção de um modelo?

### Curvas de aprendizagem

In [20]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import ShuffleSplit

#### Construa uma [curva de aprendizagem](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html) considerando o número de amostras existentes na construção do seu modelo. Se preferir use a função abaixo.

In [21]:
def plot_learning_curve(estimator, title, X, y,):
    
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
    ylim=(0.7, 1.01)
    n_jobs=1
    train_sizes=np.linspace(.1, 1.0, 5)

    plt.figure()
    plt.title(title)
    plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

#### Qual o número ideal de amostras para construção do modelo?

#### Análise o impacto do underfitting e underfitting considerando variações nos hiperparâmetros que você selecionou no processo do Grid Search. Você pode utilizar as [curvas de validação](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html). Se preferir use a função abaixo.

In [22]:
def model_complexity(estimator, title, param_name, values_param, scoring, X, y):
    """ Calculates the performance of the model as model complexity increases.
        The learning and testing errors rates are then plotted. """
    
    # Create 10 cross-validation sets for training and testing
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

    # Vary the max_depth parameter from 1 to 10

    # Calculate the training and testing scores
    train_scores, test_scores = validation_curve(estimator, X, y, \
        param_name = param_name, param_range = values_param, cv = cv, scoring = scoring)

    # Find the mean and standard deviation for smoothing
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot the validation curve
    plt.figure(figsize=(7, 5))
    ylim=(0.7, 1.01)
    plt.ylim(*ylim)
    plt.title(title)
    plt.grid()
    plt.plot(values_param, train_mean, 'o-', color = 'r', label = 'Training Score')
    plt.plot(values_param, test_mean, 'o-', color = 'g', label = 'Validation Score')
    plt.fill_between(values_param, train_mean - train_std, \
        train_mean + train_std, alpha = 0.15, color = 'r')
    plt.fill_between(values_param, test_mean - test_std, \
        test_mean + test_std, alpha = 0.15, color = 'g')
    
    # Visual aesthetics
    plt.legend(loc = 'lower right')
    plt.xlabel('Maximum Depth')
    plt.ylabel('Score')
    plt.ylim([-0.05,1.05])
    plt.show()

### Equilíbrio entre viés e variância

#### Qual o valor ideal para cada hiperparametro, ou seja, quais valores criam um modelo que melhor generaliza a predição sobre amostras futuras que são desconhecidas considerando a análise visual?

#### Quando o modelo é treinado com um valor muito baixo para o parametro definido, ele sofre com *underfitting* ou *overfitting*?
**Dica:** Como você sabe que um modelo está experimentando viés alto ou variância alta? Viés alto é um sinal de *underfitting* e alta variância é um sinal de *overfitting*. Identifique a qual parte do equilíbrio.

#### E o que acontece quando o modelo é treinado com o valor máximo? Justificar suas conclusões analisando o gráfico.

### + Experimentos ...

**Será que é necessário utilizar todas as features dos dados para atingir o valor de uma determinada KPI(Key Performance Indicator)?**

Desenvolva uma código de experimentação para analisar o impacto de cada atributo na construção de um modelo (visto até agora) considerando uma métrica definida por você e descubra qual o melhor subset de features. ***Aplique o método wrapper.***. Veja esta [dica!](https://machinelearningmastery.com/feature-selection-in-python-with-scikit-learn/)

Verifique a evolução da métrica a medida que novas features são adicionados
- Dica: Para ficar mais fácil a leitura do resultado você pode selecionar apenas as 20 primeiras features

**Selecione as outras duas soluções candidatas, e  crie uma tabela com os resultados.**

- Experimentos solução I

- Experimentos solução II

- Visualize os resultados das 3 soluções candidatas